In [2]:
from nixtlats import NixtlaClient

API key

In [5]:
from dotenv import load_dotenv
import os

# Set the TimeGPT API key
def set_api_key():
    load_dotenv()
    api_key = os.getenv("NIXTLATS_API_KEY")
    if api_key is None:
        raise ValueError("Please set the NIXTLATS_API_KEY environment variable")

nixtla_client = NixtlaClient(
    # defaults to os.environ.get("NIXTLA_API_KEY")
    api_key = set_api_key()
)

nixtla_client.validate_api_key()

In [ ]:
### data prep

In [24]:
import pandas as pd
import os

# read txt file
path = os.path.join('..','data','climate', 'AMOCdata.txt')
df = pd.read_csv(path, sep="\s+", names=["time", "AMOC0", "AMOC1", "AMOC2", "GM"], skiprows=1)

df['year'] = df['time'].apply(lambda x: int(x))
df['fraction'] = df['time'].apply(lambda x: x - int(x))

# Convert the fraction to datetime
df['date'] = pd.to_datetime(df['year'], format='%Y') + pd.to_timedelta(df['fraction'] * 365, unit='d')

# Drop the original 'time' column
df = df.drop(columns=['year', 'fraction'])

# remove clock time from date
df['date'] = df['date'].dt.date

# save the data as csv
df.to_csv(os.path.join('..','data','climate', 'AMOCdata.csv'), index=False)

<>:6: SyntaxWarning: invalid escape sequence '\s'
<>:6: SyntaxWarning: invalid escape sequence '\s'
/var/folders/ll/fp10h_1j0d1ddcld2t3r_dpr0000gn/T/ipykernel_80404/303796156.py:6: SyntaxWarning: invalid escape sequence '\s'
  df = pd.read_csv(path, sep="\s+", names=["time", "AMOC0", "AMOC1", "AMOC2", "GM"], skiprows=1)


In [16]:
df = pd.read_csv('https://raw.githubusercontent.com/Nixtla/transfer-learning-time-series/main/datasets/air_passengers.csv')
df.head()

,timestamp,value
0,1949-01-01,112
1,1949-02-01,118
2,1949-03-01,132
3,1949-04-01,129
4,1949-05-01,121


In [ ]:
nixtla_client.plot(df, time_col='time', target_col='value')